In [1]:
import os
os.environ["TF_MIN_CPP_LEVEL_LOG"] = "2"
import cv2
import time
import random
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from pathlib import Path

In [2]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D

tf.__version__

'2.5.0'

In [3]:
DATADIR = '../Dataset'
CATEGORIES = ['Cat', 'Dog']
IMG_SIZE = 64

In [4]:
def create_training_data():
    training_data = []
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=C 1=O

        for img in tqdm(os.listdir(path)):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
              
    return training_data

training_data = create_training_data()

100%|███████████████████████████████████████████████████████████████████████████| 12500/12500 [00:22<00:00, 564.07it/s]


In [5]:
print(f"data numbers: {len(training_data)}")
random.shuffle(training_data)
for sample in training_data[:5]:
    print(sample[1], end=" ")

data numbers: 24308
0 1 1 0 1 

In [6]:
X = []
Y = []

for features, label in training_data:
    X.append(features)
    Y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X/255.0

Y = np.array(Y)
X.shape

(24308, 64, 64, 3)

In [7]:
# # set up image augmentation
# from keras.preprocessing.image import ImageDataGenerator

# datagen = ImageDataGenerator(
#     rotation_range=15,
#     horizontal_flip=True,
#     width_shift_range=0.1,
#     height_shift_range=0.1
#     #zoom_range=0.3
#     )
# datagen.fit(X)

In [8]:
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=X.shape[1:]))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=128, activation='relu'))

model.add(Dense(units=2, activation = 'softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model.summary()

In [9]:
history = model.fit(X, Y, batch_size=64, epochs=100, validation_split=0.1)
# model.fit_generator(datagen.flow(X, Y, batch_size=32),
#                     epochs=100,
#                     verbose=1)


Epoch 1/100
342/342 [==============================] - 21s 59ms/step - loss: 0.6757 - accuracy: 0.5610 - val_loss: 0.6557 - val_accuracy: 0.6179
Epoch 2/100
342/342 [==============================] - 20s 59ms/step - loss: 0.5962 - accuracy: 0.6772 - val_loss: 0.5824 - val_accuracy: 0.6911
Epoch 3/100
342/342 [==============================] - 20s 59ms/step - loss: 0.5624 - accuracy: 0.7057 - val_loss: 0.5581 - val_accuracy: 0.7063
Epoch 4/100
342/342 [==============================] - 20s 59ms/step - loss: 0.5426 - accuracy: 0.7213 - val_loss: 0.5407 - val_accuracy: 0.7244
Epoch 5/100
342/342 [==============================] - 20s 59ms/step - loss: 0.5288 - accuracy: 0.7304 - val_loss: 0.5305 - val_accuracy: 0.7330
Epoch 6/100
342/342 [==============================] - 20s 59ms/step - loss: 0.5224 - accuracy: 0.7358 - val_loss: 0.5151 - val_accuracy: 0.7413
Epoch 7/100
342/342 [==============================] - 20s 59ms/step - loss: 0.5118 - accuracy: 0.7439 - val_loss: 0.5476 - val_ac

342/342 [==============================] - 20s 58ms/step - loss: 0.2835 - accuracy: 0.8766 - val_loss: 0.3768 - val_accuracy: 0.8313
Epoch 58/100
342/342 [==============================] - 20s 58ms/step - loss: 0.2831 - accuracy: 0.8782 - val_loss: 0.3827 - val_accuracy: 0.8334
Epoch 59/100
342/342 [==============================] - 20s 58ms/step - loss: 0.2795 - accuracy: 0.8815 - val_loss: 0.3911 - val_accuracy: 0.8276
Epoch 60/100
342/342 [==============================] - 20s 58ms/step - loss: 0.2746 - accuracy: 0.8828 - val_loss: 0.3982 - val_accuracy: 0.8174
Epoch 61/100
342/342 [==============================] - 20s 58ms/step - loss: 0.2726 - accuracy: 0.8878 - val_loss: 0.3899 - val_accuracy: 0.8371
Epoch 62/100
342/342 [==============================] - 20s 58ms/step - loss: 0.2741 - accuracy: 0.8842 - val_loss: 0.3955 - val_accuracy: 0.8248
Epoch 63/100
342/342 [==============================] - 20s 58ms/step - loss: 0.2701 - accuracy: 0.8852 - val_loss: 0.3765 - val_accuracy

In [10]:
model.save('../Models/20210701.h5')

In [11]:
# 单个测试样本数据
test_path = "../Images/test.jpg"
image = cv2.imread(test_path)
image = cv2.resize(image, (64, 64))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_bn = image.astype("float32") / 255.0
image = np.expand_dims(image, axis=0)
image_bn = np.expand_dims(image_bn, axis=0)

In [12]:
# 恢复 keras 模型，并预测
keras_file = '../Models/20210701.h5'
model = tf.keras.models.load_model(keras_file)
# model.summary()
# tf.autograph.set_verbosity(0)

start_time = time.time()
pred = model.predict(image_bn)
stop_time = time.time()

print(f"prediction: {pred}")
print('time: {:.3f}ms'.format((stop_time - start_time) * 1000))
print("model size: {:.2f} MB".format(os.path.getsize(keras_file)/1024/1024))

prediction: [[9.9983859e-01 1.6139554e-04]]
time: 82.779ms
model size: 7.47 MB


In [13]:
# plot_model(model, to_file='model_small.png', show_layer_names=False, show_shapes=True)

In [14]:
# quantize int

def representative_data_gen():
  for input_value in X[:100]:
    input_value = np.expand_dims(input_value, axis=0)
    input_value = input_value.astype(np.float32)
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error

tflite_model = converter.convert()

tflite_file = Path("../Models/Tflites/CDC_half.tflite")
tflite_file.write_bytes(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Sheat\AppData\Local\Temp\tmpl4i8k_dc\assets


656360

In [15]:
# tflite 模型推理

interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

interpreter.set_tensor(input_details['index'], image_bn)

start_time = time.time()
interpreter.invoke()
stop_time = time.time()

output_data = interpreter.get_tensor(output_details['index'])
print(f"prediction: {output_data}")
print('time: {:.3f}ms'.format((stop_time - start_time) * 1000))
print("model size: {:.2f} MB".format(os.path.getsize(tflite_file)/1024/1024))

prediction: [[0.99609375 0.        ]]
time: 56.847ms
model size: 0.63 MB


In [16]:
print(X[:100].shape)

(100, 64, 64, 3)


In [17]:
# quantize int

def representative_data_gen():
  for input_value in X[:100]:
    input_value = np.expand_dims(input_value, axis=0)
    input_value = input_value.astype(np.float32)
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model = converter.convert()

tflite_file = Path("../Models/Tflites/CDC.tflite")
tflite_file.write_bytes(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Sheat\AppData\Local\Temp\tmpchtknqml\assets


INFO:tensorflow:Assets written to: C:\Users\Sheat\AppData\Local\Temp\tmpchtknqml\assets


656432

In [18]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [19]:
# tflite 模型推理
tflite_file = Path("../Models/Tflites/CDC.tflite")
interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

interpreter.set_tensor(input_details['index'], image)

start_time = time.time()
interpreter.invoke()
stop_time = time.time()

output_data = interpreter.get_tensor(output_details['index'])
print(f"prediction: {output_data}")
print('time: {:.3f}ms'.format((stop_time - start_time) * 1000))
print("model size: {:.2f} MB".format(os.path.getsize(tflite_file)/1024/1024))

prediction: [[255   0]]
time: 56.848ms
model size: 0.63 MB
